# Mesure de concentrations de gaz 

Les instruments de mesure de concentration gazeuses de la famille MQ  fournissent un signal analogique c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

## Utilisation de convertisseurs Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique
 * ADS1015 16bit
 * ADS1115 12 bit
dont le fonctionnement est totalement identique. Ce qui les diffférencie est la grandeur privilégiée, pour le premier c'est la précision de l'échantillonnage aux dépens de la fréquence d'échantillonnage pour le second c'est l'inverse.

Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie
Nous disposons de 3 capeurs et de 4 broches d'échantillonnage sur le convertisseur, nous ferons deux mesures directes et une mesure par différence. L'exploitation fine de ces éléments requiert le calibrage des capteurs analogiques et le choix des bons paramètres de configuration. L'exemple simple présenté ici a été réalisé en mode direct pour le capteur qualité et le capteur aclcool en mode différentiel pour le capteur méthane.

La figure ci-dessous présente l'ensemble du dispositif :

![Convertisseurs vue générale](images/montageGaz.jpg)
                        ![licence : Gérard Vidal](../images/Licence.jpg)
                        
Le câblage du convertisseurs ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |

les 3 capteurs utilisés sont :
 * Qualité de l'air 
 * Alcool
 * Méthane

L'image suivante montre la connexion des trois capteurs de gaz : Qualité, Méthane et Alcool :

![Convertisseurs vue du câblage](images/3capteursGaz.jpg)
                        ![licence : Gérard Vidal](../images/Licence.jpg)

Connexion du capteur méthane :

![Capteurs gaz connectés au convertisseur](images/convertisseur.jpg)
                        ![licence : Gérard Vidal](../images/Licence.jpg)
                        
Importation des librairies nécessaires à l'utilisation du convertisseur. On n'importe aucune librairie propre aux capteurs de gaz car il s'agit de capteurs `analogiques` qui transmettent une variation de potentiel sur un fil série.

In [53]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer.
 * getGain()
 * getSPS()
 * getContinuous()

In [54]:
gaz = upm.ADS1115(0, 0x49)
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), "Continuous : {0:5d}".format(cont))
gaz.setContinuous(True) # Pour ne pas faire de conversion en continu
gaz.setGain(upm.ADS1X15.GAIN_TWO) # Passage à un gain 1024
gaz.setSPS(upm.ADS1115.SPS_32) # Passage à une fréquence de 64
# Vérification
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :    32 Continuous :     1

Nouvelles valeurs :
Gain :  1024  SPS :    64 Continuous :     1


## Mesure directe des variables qualité alcool méthane

Les deux premières mesures sont effectués directement, la troisième est faite par différence.

In [55]:
qual = ads1115.getSample(upm.ADS1X15.SINGLE_0)
alco = ads1115.getSample(upm.ADS1X15.SINGLE_1)
meth = ads1115.getSample(upm.ADS1X15.DIFF_2_3)
print ("Qualité : {0:.2f}\n".format(qual),"Alcool : {0:.2f}\n".format(alco),"Méthane : {0:.2f}\n".format(meth))

Qualité : 0.80
 Alcool : 0.80
 Méthane : 2.50



# série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 20 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [51]:
for num in range(0,20):
    qual = ads1115.getSample(upm.ADS1X15.SINGLE_0)
    alco = ads1115.getSample(upm.ADS1X15.SINGLE_1)
    meth = ads1115.getSample(upm.ADS1X15.DIFF_2_3)
    print ("Qualité : {0:.2f} ".format(qual),"Alcool : {0:.2f} ".format(alco),"Méthane : {0:.2f} ".format(meth))    
    sleep(2)

Qualité : 0.08  Alcool : 2.32  Méthane : 0.08 
Qualité : 0.08  Alcool : 2.29  Méthane : 2.95 
Qualité : 0.08  Alcool : 2.26  Méthane : 2.93 
Qualité : 0.08  Alcool : 2.24  Méthane : 2.93 
Qualité : 0.08  Alcool : 2.20  Méthane : 2.89 
Qualité : 0.08  Alcool : 2.16  Méthane : 2.87 
Qualité : 0.08  Alcool : 2.14  Méthane : 2.88 
Qualité : 0.08  Alcool : 2.14  Méthane : 2.90 
Qualité : 0.08  Alcool : 2.13  Méthane : 2.89 
Qualité : 0.08  Alcool : 2.11  Méthane : 2.89 
Qualité : 0.08  Alcool : 2.10  Méthane : 2.88 
Qualité : 0.08  Alcool : 2.07  Méthane : 2.87 
Qualité : 0.08  Alcool : 2.05  Méthane : 2.86 
Qualité : 0.08  Alcool : 2.84  Méthane : 2.84 
Qualité : 0.08  Alcool : 2.00  Méthane : 2.83 
Qualité : 0.08  Alcool : 2.00  Méthane : 2.85 
Qualité : 1.98  Alcool : 1.98  Méthane : 2.83 
Qualité : 0.08  Alcool : 1.96  Méthane : 2.81 
Qualité : 0.08  Alcool : 1.94  Méthane : 2.81 
Qualité : 0.08  Alcool : 1.92  Méthane : 2.80 
